# Overview

This notebook provides some tips and tricks for *data wrangling*-- the art of turning messy "real world" data into the sort of data you can actual use.  We'll explore several example datasets:
- A "college scorecard" dataset (`data-stories/college-scorecard`)
- A "nutrition" dataset (`data-stories/nutrition-data`)

We'll be using the `pandas` library.

In [1]:
import pandas as pd
import numpy as np

# Loading in data

The `pandas` library can read from a wide variety of file types and formats.  A full list of supported formats may be found [here](https://pandas.pydata.org/pandas-docs/stable/reference/io.html).  We will be exploring two common data formats: a CSV (comma separated value) file and an XLSX (Excel spreadsheet) file.

In [2]:
college_data = pd.read_csv('../data-stories/college-scorecard/scorecard_data.csv')

## A messy dataframe
When raw data is read into a `DataFrame`, by default each row of the file (exept the first) gets a row of the dataframe, indexed with the integers 0 through $n$ (where $n$ is the number of rows in the file, excluding the first). Each column is named according to the first line of the file (called a "header" line).

In [3]:
college_data.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,INSTURL,NPCURL,HCM2,PREDDEG,...,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GT_25K_P6,GT_28K_P6,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP
0,100654,100200,1002,Alabama A & M University,Normal,AL,www.aamu.edu/,www2.aamu.edu/scripts/netpricecalc/npcalc.htm,0,3,...,0.7667,0.0859,31000,0.453,0.431,32750,348.16551225731,0.2531554273,NaN,0.2913
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,www.uab.edu,uab.studentaidcalculator.com/survey.aspx,0,3,...,0.5179,0.2363,41200,0.669,0.631,21833,232.106797835537,0.513963161,NaN,0.5384
2,100690,2503400,25034,Amridge University,Montgomery,AL,www.amridgeuniversity.edu,www2.amridgeuniversity.edu:9091/,0,3,...,0.8436,0.8571,39600,0.658,0.542,22890,243.343773299842,0.2307692308,NaN,PrivacySuppressed
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,finaid.uah.edu/,0,3,...,0.4312,0.2255,46700,0.685,0.649,22647,240.760438353933,0.5485090298,NaN,0.4905
4,100724,100500,1005,Alabama State University,Montgomery,AL,www.alasu.edu,www.alasu.edu/cost-aid/forms/calculator/index....,0,3,...,0.8113,0.0974,27700,0.393,0.351,31500,334.876752247489,0.2185867473,NaN,0.2475


## Basic cleanup

In this example file, let's perform some basic cleanup:
  - Set the index to the `UNITID` field
  - Drop the following columns: OPEID, OPEID6, NPCURL
  - Replace non-numeric values in the HCM2 column or later with np.nan
  - Rename columns to be more human-readable

In [4]:
college_data.set_index('UNITID', inplace=True) #set the index to the values in the UNITID field
college_data.drop(['OPEID', 'OPEID6', 'NPCURL'], inplace=True, axis=1) #drop some columns

In [5]:
#converts non-numeric values to NaNs
# - loc selects out a subset of the rows and columns of the dataframe.
#   ":" selects all the rows, and "'HCM2':" selects all of the columns after (and including) "HCM2"
# - apply runs a function on each entry of the dataframe and sets the value at the given row/column to whatever
#   that function outputs (when run on that row/column's data)
# - lambda x: pd.to_numeric(x, errors='coerce') is a shorthand notation that defines a function that converts the
#   given input to a (scalar) numerical value, and if it fails to convert correctly it sets the value to np.nan
# - the left side of the expression indicates which values of the college_data dataframe to change.  The right
#   side of the expression indicates what the new values will be

college_data.loc[:, 'HCM2':] = college_data.loc[:, 'HCM2':].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [6]:
college_data.head()

,INSTNM,CITY,STABBR,INSTURL,HCM2,PREDDEG,HIGHDEG,CONTROL,LOCALE,HBCU,...,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GT_25K_P6,GT_28K_P6,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,Normal,AL,www.aamu.edu/,0,3,4,1,12.0,1.0,...,0.7667,0.0859,31000.0,0.453,0.431,32750.0,348.165512,0.253155,NaN,0.2913
100663,University of Alabama at Birmingham,Birmingham,AL,www.uab.edu,0,3,4,1,12.0,0.0,...,0.5179,0.2363,41200.0,0.669,0.631,21833.0,232.106798,0.513963,NaN,0.5384
100690,Amridge University,Montgomery,AL,www.amridgeuniversity.edu,0,3,4,2,12.0,0.0,...,0.8436,0.8571,39600.0,0.658,0.542,22890.0,243.343773,0.230769,NaN,NaN
100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,0,3,4,1,12.0,0.0,...,0.4312,0.2255,46700.0,0.685,0.649,22647.0,240.760438,0.548509,NaN,0.4905
100724,Alabama State University,Montgomery,AL,www.alasu.edu,0,3,4,1,12.0,1.0,...,0.8113,0.0974,27700.0,0.393,0.351,31500.0,334.876752,0.218587,NaN,0.2475


In [7]:
#code for printing out the column names
np.array(college_data.columns)

array(['INSTNM', 'CITY', 'STABBR', 'INSTURL', 'HCM2', 'PREDDEG',
       'HIGHDEG', 'CONTROL', 'LOCALE', 'HBCU', 'PBI', 'ANNHI', 'TRIBAL',
       'AANAPII', 'HSI', 'NANTI', 'MENONLY', 'WOMENONLY', 'RELAFFIL',
       'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75',
       'SATVRMID', 'SATMTMID', 'SATWRMID', 'ACTCM25', 'ACTCM75',
       'ACTEN25', 'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTWR25', 'ACTWR75',
       'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'ACTWRMID', 'SAT_AVG',
       'SAT_AVG_ALL', 'PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09',
       'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15',
       'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25',
       'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38',
       'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44',
       'PCIP45', 'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50',
       'PCIP51', 'PCIP52', 'PCIP54', 'DISTANCEONLY', 'UGDS', 'UGDS_WHITE',
       'UGDS_BLACK', 'UGDS_HISP',

In [8]:
#create a dictionary that specifies which columns to rename and what to rename them to
names = {'INSTNM': 'Institution',
         'CITY': 'City',
         'STABBR': 'State',
         'INSTURL': 'Website'}
college_data.rename(names, inplace=True, axis=1) #rename the given columns (specified in the dictionary)

In [9]:
#print out the cleaned-up dataframe
college_data.head()

,Institution,City,State,Website,HCM2,PREDDEG,HIGHDEG,CONTROL,LOCALE,HBCU,...,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GT_25K_P6,GT_28K_P6,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_L4_POOLED_SUPP,C150_4_POOLED_SUPP
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,Normal,AL,www.aamu.edu/,0,3,4,1,12.0,1.0,...,0.7667,0.0859,31000.0,0.453,0.431,32750.0,348.165512,0.253155,NaN,0.2913
100663,University of Alabama at Birmingham,Birmingham,AL,www.uab.edu,0,3,4,1,12.0,0.0,...,0.5179,0.2363,41200.0,0.669,0.631,21833.0,232.106798,0.513963,NaN,0.5384
100690,Amridge University,Montgomery,AL,www.amridgeuniversity.edu,0,3,4,2,12.0,0.0,...,0.8436,0.8571,39600.0,0.658,0.542,22890.0,243.343773,0.230769,NaN,NaN
100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,0,3,4,1,12.0,0.0,...,0.4312,0.2255,46700.0,0.685,0.649,22647.0,240.760438,0.548509,NaN,0.4905
100724,Alabama State University,Montgomery,AL,www.alasu.edu,0,3,4,1,12.0,1.0,...,0.8113,0.0974,27700.0,0.393,0.351,31500.0,334.876752,0.218587,NaN,0.2475


# Exercise
Play around with the food_data DataFrame and clean it up as follows:
- Set the index to NDB_No
- Change the formatting of the Shrt_Desc field to include spaces after the commas (hint: use apply and the str.replace function)
- Drop columns containing "GmWt_Desc" in the name
- For any renaiming column with any of the following sub-strings in the name, replace all non-numerical values with np.nan:
  - (µg)
  - (g)
  - (mg)
  - GmWt_
- Rename some of the columns to have more human-readable names (documentation may be found in the data-stories/nutrition-data folder)

Use the head command to print out the first 5 rows of the dataframe before and after your changes.

In [10]:
food_data = pd.read_excel('../data-stories/nutrition-data/food_data.xlsx')

In [11]:
# add your code here!